# Intro to Statistical Analysis with Python

**Date**: November 7th, 2024

**Author**: Sunny Fang, yf2610

_Created as part of the Barnard College Computing Fellows Program, Fall 2024_


By the end of this workshop, students should be able to...
1. Write well-documented, interpretable code for statistical testing,
2. Interpret and explain statistical test results to an audience,
3. Feel comfortable selecting and implementing the appropriate statistical test for their project,

# 0. Setup

In [ ]:
%pip install pandas
%pip install seaborn
%pip install --upgrade matplotlib
%pip install scipy

In [2]:
# to process data
import pandas as pd

# for numerical processing
import numpy as np

# to plot data
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

# to carry out statistical testing
from scipy import stats

sns.set_style('darkgrid')

# for datetime processing
import datetime as dt

# 1. Data Preprocessing

Today, we are using an air pollution dataset from [Kaggle](https://www.kaggle.com/datasets/sogun3/uspollution/data). 

You can read the file with the following link: 

In [ ]:
# TODO: replace the filename
# read the dataset using the link provided above
df = pd.read_csv("___")

# remove max column restriction
pd.set_option("display.max_columns", None)

# display the first 5 rows
df.head(5)

In [ ]:
df_cols = df.columns.to_list()
print(df_cols)

In [ ]:
# Carry out basic data preprocessing techniques
# step 1: drop NaNs
df = df.dropna()

# step 2: drop columns we don't need
# TODO: keep everything *except for* the first 5 columns
# i.e., we don't need: Unnamed: 0, State Code,County Code, Site Num, and Address
# there are several right answers to this!
df = ___
df.head()


In [6]:
# an important addition: as a good practice, when dealing with datetime objects in a dataframe, 
df['Date Local'] = pd.to_datetime(df['Date Local'])

# 2. T-test

### 2.1 One-sample T-test

On October 1, 2015, the Environmental Protection Agency strengthend the National Ambient Air Quality Standards: "areas will meet the standards if the 4th highest daily maximum 8-hour ozone concentration per year, averaged over three years, is **equal to or less than 70 ppb** (or 0.07 ppm) ([source](https://19january2017snapshot.epa.gov/ozone-pollution/2015-revision-2008-ozone-national-ambient-air-quality-standards-naaqs-supporting_.html)).

Here, we want to test whether or not the state of Califronia (CA) meets the standard. For the purpose of this demonstration, we are going to loosen the "4th highest daily maximum 8-hour ozone concentration per year" assumption. Instead, we are going to see if the **average of the max O3 value across three years** meet the standards.

**Resources**:
- [How to filter by year](https://stackoverflow.com/questions/46878156/pandas-filter-dataframe-rows-with-a-specific-year)


#### step 1: **define hypotheses**

Before we jump into data analysis, take a moment and formulate our null and alternative hypotheses:

- Null hypothesis ($H_0$): The mean value of the average daily ozone concentration in CA is equal to 0.07. 

- Alternative Hypothesis ($H_A$): The mean value of the average daily ozone concentration in CA is larger than 0.07. 

#### step 2: **define $\alpha$ (significance level)**

Typically, $\alpha$ is set to be 0.05.

#### step 3: **data slicing**

In [ ]:
# TODO: let's subset the data together!
# step 1: subset the data to get our desired output
# note: here, we subset for "O3 1st Max Value", 
# but we can also use "O3 Mean" depending on our research question
# 1a: subset the dataset, keeping California only, name it 'ca'
ca = df[df['State'] == '__']

# 1b: subset 'ca' to keep data where year is after 2011 (inclusive)
ca_2011 = ca[ca['Date Local'].dt.year >= ___]

# 1c: subset 'ca_2011' to keep data where year is before 2013 (inclusive)
ca_2011_2013 = ca_2011[ca_2011['Date Local'].dt.year <= ___]

# 1d: last but not least, we can just keep the columns we want
ca_2011_2013 = ca_2011_2013[['Date Local', 'O3 1st Max Value']]

# alternatively, we can write all of this in one line!
# ca_2011_2013 = df[(df['State'] == 'California') & ((df['Date Local'].dt.year >= 2011) & (df['Date Local'].dt.year <= 2013))][['Date Local','O3 1st Max Value']]

# optional but highly recommended: rename columns for easier access
ca_2011_2013 = ca_2011_2013.rename(columns={"Date Local": "date", "O3 1st Max Value": "o3"})

# always a good idea to check the data
display(ca_2011_2013.head())

# save file
ca_2011_2013.to_csv("../data/ca_2011_2013.csv")

#### step 4: **check assumptions**

Usually in t-tests, there are several assumptions that should be followed:
- Independence of samples: samples should be randomly selected
- Identically Distributed: samples should come from the same distribution
- Normality (or sample size >30): samples should be normally distributed (Q-Q plots)
- Equal variances: samples should have equal variances (box plots)

In our example, we are conducting a one-sample t-test, so only the first assumption applies. However, due to the nature of our data being collected over time, we are going to loosen the assumption that time has an effect on ground Ozone levels.

Resources:
- [Tutorial on t-tests](https://www.datacamp.com/tutorial/an-introduction-to-python-t-tests)

#### optional step: **data visualization**

As seen earlier in the workshop, visualizations can be powerful! Here, we demonstrate some ways we can create visualizations for this question. You can find a section in the end of this note book for skeleton code used for plotting.

In [ ]:
# TODO: live coding!
# for our question, we can plot... 
# (1) histogram to see the distribution - directly related to hypothesis testing
# (2) scatterplot to see dates with O3 values that exceed the standard

# first, declare the figure
fig, ax = plt.subplots(2, 1, figsize=(16, 8), dpi=400)
plt.tight_layout(pad=4.0)

# next, plot with Seaborn
# want to see distribution of o3
sns.____(x="__", 
             data=ca_2011_2013, 
             label = "O3 1st Max Value",
             ax=ax[0])

# want to see exceedance across date
sns.___(x="date", 
                y="__", 
                data=ca_2011_2013, 
                label="O3 1st Max Value",
                ax=ax[1])

sns.___(x="date", 
                y="o3", 
                data=ca_2011_2013[ca_2011_2013['o3']>0.07], 
                label="Above threshold", 
                color = 'r',
                ax=ax[1])

# don't forget to set labels and titles
ax[0].set_xlabel("Ozone 1st max value")
ax[0].set_ylabel("Frequency count")
ax[0].set_title("Distribution of Ozone in California, 2011-2013", 
                fontsize = 16)

ax[1].set_xlabel("Date")
ax[1].set_ylabel("Ozone 1st max value")
ax[1].set_title("Ozone Levels in California (2011-2013): Daily Observations with Exceedances Highlighted", 
                fontsize = 16)

# additional labels
ax[0].axvline(x=0.07, linewidth=3, color='r', label = "threshold")
ax[0].axvline(x=ca_2011_2013['o3'].mean(), linewidth=3, color='b', label = "mean")
ax[0].legend(loc=0, fontsize=14)

ax[1].axhline(y=0.07, linewidth=3, color='r', linestyle='--')
ax[1].legend(loc=0, fontsize=14)

# optional: add caption
# caption = "your caption here"
# fig.text(x, y, caption, ha='center')

# save figure
plt.savefig("figures/ca_o3_eda.png")
# if using Google colab:
# files.download ("filename.png")

#### step 5: **statistical testing**

In [ ]:
# import 1 sample t-test from scipy
from scipy.stats import t
from scipy.stats import ttest_1samp

# peform one-tailed one-sample t-test
x = ca_2011_2013['o3']
t_stat, p_value = ttest_1samp(a = x, 
                              popmean = 0.070,
                              alternative="greater")
print(f"One-sample t-Test results: \nt-statistic = {t_stat:0.3f} \np-value = {p_value}")

In [ ]:
x = ca_2011_2013['o3']
dof = len(x) - 1
x_axis = np.linspace(-5, 5, 100)
y = t.pdf(x_axis, dof)

alpha = 0.05

# Calculate the critical t-value for the right-tailed test
critical_value = t.ppf(1 - alpha, dof)

# Create the plot
fig, ax = plt.subplots(1, 1, figsize=(8, 4), dpi=200)
plt.plot(x_axis, y, label='t-distribution', color='olive')

# Shade the rejection region
x_fill = np.linspace(critical_value, 5, 100)
y_fill = t.pdf(x_fill, dof)
plt.fill_between(x_fill, y_fill, color='khaki', alpha=0.5, label='Rejection Region')
import random
x = [random.uniform(0.07, 0.071) for i in range(3)]
test_statistic, p_value = ttest_1samp(x, 0.06, alternative="greater")
print(test_statistic)

# Add a vertical line at the critical value
plt.axvline(x=critical_value, color='darkgoldenrod', linestyle='--', label=f'Critical Value (t={critical_value:.2f})')

# Add labels and title
plt.title('t-Distribution with Rejection Region for Right-Tailed Test')
plt.xlabel('t-value')
plt.ylabel('Probability Density')
plt.legend()
plt.show()

#### step 6: **statistical testing**

In [11]:
# TODO: Write your conclusion in the Markdown box below

Since the p-value is 1.0 and thus not less than 0.05, it means we do not have sufficient evidence to reject the null hypothesis.

#### optional step: **more data visualization**

We can also visualize the confidence intervals of the samples.

In [ ]:
# first, declare the figure
fig, ax = plt.subplots(1, 1, figsize=(16, 4), dpi=400)
plt.tight_layout(pad=3.0)

# next, plot with Seaborn
sns.histplot(x="o3", 
             data=ca_2011_2013, 
             label = "O3 1st Max Value",
             ax=ax)

# don't forget to set labels and titles
ax.set_xlabel("Ozone 1st max value")
ax.set_ylabel("Frequency count")
ax.set_title("Distribution of Ozone in California, 2011-2013", fontsize = 14)

# additional labels
threshold = 0.07
ci_higher = ca_2011_2013.o3.quantile(0.95)
ax.axvline(x=threshold, linewidth=2, color='r', label = "threshold")
ax.axvline(x=ci_higher, linewidth=2, color='b', label = "confidence interval")
plt.legend(loc=0)
# save figure
plt.savefig("figures/ci_and_h0.png")

#### **IMPORTANT NOTE**: 
There are several variations to the t-test, such as two-sample t-tests (student's t-test and Welch's t-test for unequal variance). The most important skill here for research is:
1. Look at your data and formulate a research question.
2. Decide the most appropriate statistical test to apply.

If you ever have doubts, always ask! :)

# Appendix 1. Visualization Recipe

Below is a skeleton code for visualizing with Seaborn and Matplotlib, but this is NOT meant for you to follow the format stringently. 
```
# skeleton code for plotting in python

# first, declare the figure
fig, ax = plt.subplots(n_row, n_col, figsize = (width, height), dpi = 400)

# next, plot with Seaborn
# note: change the ax=ax[i] argument if you have multiple subplots
sns.histplot(x=iv, data=data, ax=ax)
sns.lineplot(x=iv, y=dv, data=data, label="legend label", ax=ax)
sns.barplot(x=iv, y=dv, data=data, ax=ax)

# don't forget to set labels and titles
# similarly, change ax to ax[i] if you have multiple subplots
ax.set_xlabel("your xlabel here")
ax.set_ylabel("your ylabel here")
ax.set_title("your title here")

# optional: add caption
# similarly, change ax to ax[i] if you have multiple subplots
caption = "your caption here"
ax.text(x, y, caption, ha='center')

# save figure
plt.savefig("filename.png")
# if using Google colab:
files.download("filename.png")
```